Global Averages Example
=========

**Summary:**   Full-volume averages of requested output variables over the full, spherical shell.  These files store a time-series of the specified output quantities $q_i(t)$ such that:

$q_i(t)=\frac{1}{V}\int q_i(r,\theta,\phi,t)dV$

**Subdirectory:**  G_Avgs

**main_input prefix:** globalavg

**Python Class:** G_Avgs

**Additional Namelist Variables:**  
None

*Before proceeding, ensure that you have copied Rayleigh/post_processing/rayleigh_diagnostics.py to your simulation directory.  This Python module is required for reading Rayleigh output into Python.*

Examining the *main_input* file, we see that the following output values have been denoted for the Global Averages (see *rayleigh_output_variables.pdf* for the mathematical formulae):


| Menu Code | Description |
|-----------|-------------|
| 401       | Full Kinetic Energy Density (KE) |
| 402       | KE (radial motion) |
| 403       | KE (theta motion)  |
| 404       | KE (phi motion) |
| 405       | Mean Kinetic Energy Density (MKE) |
| 406       | MKE (radial motion) |
| 407       | MKE (theta motion) |
| 408       | MKE (phi motion) |
| 409       | Fluctuating Kinetic Energy Density (FKE) |
| 410       | FKE (radial motion) |
| 411       | FKE (theta motion) |
| 412       | FKE (phi motion) |

In the example that follows, we will plot the time-evolution of these different contributions to the kinetic energy budget.  We begin with the following preamble:

In [ ]:
%matplotlib inline
from rayleigh_diagnostics import G_Avgs, build_file_list
import matplotlib.pyplot as plt
import numpy
import warnings
warnings.filterwarnings("ignore")


model_type = 1   # 1 for anelastic example, 2 Boussinesq example
saveplot = True # Set to True to save plots generated in this notebook to pdf
font_size=14     # Font size for plot labels 

base_dir = '/rayleigh_tutorial/'

# Define some model-specific information
if (model_type == 1):
    model_dir = base_dir+'anelastic/'
    eunits = r'(erg cm$^{-3}$)' # energy density
    tunits = '(s)'              # time
    imin = 0             # minimum iteration number to process for time series
    imax = 10000000      # maximum iteration number to process for time series
    
if (model_type == 2):
    model_dir = base_dir+'Boussinesq/'
    eunits = ''                          # energy density
    tunits = '(viscous diffusion times)' # time
    imin = 0             # minimum iteration number to process for time series
    imax = 10000000      # maximum iteration number to process for time series

The preamble for each plotting example will look similar to that above.  We import the numpy and matplotlib.pyplot modules, aliasing the latter to *plt*.   We also import two items from *rayleigh_diagnostics*: a helper function *build_file_list* and the *GlobalAverage* class. 

The *G_Avgs* class is the Python class that corresponds to the full-volume averages stored in the *G_Avgs* subdirectory of each Rayleigh run.

We will use the build_file_list function in many of the examples that follow.  It's useful when processing a time series of data, as opposed to a single snapshot.  This function accepts three parameters: a beginning time step, an ending time step, and a subdirectory (path).  It returns a list of all files found in that directory that lie within the inclusive range [beginning time step, ending time step].  The file names are prepended with the subdirectory name, as shown below.

In [ ]:
# Build a list of all files ranging from iteration 0 million to 10 million
files = build_file_list(imin,imax,path=model_dir+'G_Avgs')
print(files)

We can create an instance of the G_Avgs class by initializing it with a filename.  The optional keyword parameter *path* is used to specify the directory.  If *path* is not specified, its value will default to the subdirectory name associated with the datastructure (*G_Avgs* in this instance).  I.e., the default is to look for files in a subdirectory of the directoy from within which the notebook is being run.

Each class was programmed with a **docstring** describing the class attributes.   Once you created an instance of a rayleigh_diagnostics class, you can some helpful information regarding its attributes and methods using the help function as shown below.

In [ ]:
ga = G_Avgs(filename=files[0],path='') 
help(ga)

Examining the docstring, we see a few important attributes that are common to the other Rayleigh output types:
1.  niter -- the number of time steps in the file
2.  nq    -- the number of output variables stored in the file
3.  qv    -- the menu codes for those variables
4.  vals  -- the actual data
5.  time  -- the simulation time corresponding to each timestep in the file

The first step in plotting a time series is to collate the data.

This is easy to do with the G_Avgs class. Instead of passing it a single file, we pass it a list of files and the concatenation of the data is handled automatically via the initialization routines.

In [ ]:
ga = G_Avgs(files,path='')
ga.niter
gavgs = ga.vals
time  = ga.time
iters = ga.iters

The Lookup Table (LUT)
------------------

The next step in the process is to identify where within the *gavgs* array our deisired output variables reside.  Every Rayleigh file object possesses a lookup table (lut).  The lookup table is a python list used to identify the index within the vals array where a particular menu code resides.  For instance, the menu code for the theta component of the velocity is 2.  The location of v_theta in the vals array is then stored in lut[2].  

Note that you should never assume that output variables are stored in any particular order.  Moreover, the lookup table is unique to each file and is likely to change during a run if you modify the output variables in between restarts.  When running the benchmark, we kept a consistent set of outputs throughout the entirety of the run.  This means that the lookup table did not change between outputs and that we can safely use the final file's lookup table (or any other file's table) to reference our data.

Plotting Kinetic Energy
---------------------------
Let's examine the different contributions to the kinetic energy density in our models.  Before we can plot, we should use the lookup table to identify the location of each quantity we are interested in plotting.

In [ ]:
#The indices associated with our various outputs are stored in a lookup table
#as part of the GlobalAverage data structure.  We define several variables to
#hold those indices here:

lut = ga.lut
ke  = lut[401]  # Kinetic Energy (KE)
rke = lut[402]  # KE associated with radial motion
tke = lut[403]  # KE associated with theta motion
pke = lut[404]  # KE associated with azimuthal motion

#We also grab some energies associated with the mean (m=0) motions
mke  = lut[405]
mrke = lut[406]  # KE associated with mean radial motion
mtke = lut[407]  # KE associated with mean theta motion
mpke = lut[408]  # KE associated with mean azimuthal motion

#We also output energies associated with the fluctuating/nonaxisymmetric
#motions (e.g., v- v_{m=0})
fke  = lut[409]
frke = lut[410]  # KE associated with mean radial motion
ftke = lut[411]  #KE associated with mean theta motion
fpke = lut[412]  # KE associated with mean azimuthal motion

To begin with, let's plot the total, mean, and fluctuating kinetic energy density during the initial transient phase, and then during the equilibrated phase.

In [ ]:
sizetuple=(10,4)
fig, ax = plt.subplots( figsize=sizetuple)
plt.rcParams.update({'font.size': font_size})

ax.plot(time, gavgs[:,ke], label='KE')
ax.plot(time, gavgs[:,mke], label = 'MKE')
ax.plot(time,gavgs[:,fke],label='FKE')
ax.legend(loc='center right', shadow=True)
ax.set_title('Volume-Averaged Energy Density')
ax.set_xlabel('Simulation Time '+tunits)
ax.set_ylabel('Energy Density '+eunits)

plot_file1 ='energy_vs_time_1.pdf'  
plt.tight_layout()
if (saveplot):    
    plt.savefig(plot_file1)
else:
    plt.show()

We can also look at the energy associated with each velocity component.

In [ ]:
sizetuple=(10,10)
xlims=[0,0.2]
qinds = [ [ke,rke,tke,pke] , [fke, frke, ftke, fpke] , [mke, mrke, mtke, mpke] ]
labels = ['Total', 'r', r'$\theta$', r'$\phi$']
titles = ['','Fluctuating ', 'Mean ']

fig, ax = plt.subplots(ncols=1, nrows=3, figsize=sizetuple)
for i in range(3):
    for j in range(4):
        ax[i].plot(time, gavgs[:,qinds[i][j]], label=labels[j])

        ax[i].legend(loc='center right', shadow=True, ncol=2)
        ax[i].set_title(titles[i]+'Energy Density Breakdown')
        ax[i].set_xlabel('Time '+tunits)
        ax[i].set_ylabel('Energy Density '+eunits)


plot_file2 ='energy_vs_time_2.pdf'
plt.tight_layout()

if (saveplot):    
    plt.savefig(plot_file2)
else:
    plt.show()